In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [2]:
np.random.seed(1)
index=np.array([i for i in range(5612)])
np.random.shuffle(index)

#we'll take the first 300 as the test set, next 300 as the dev set, and the last remainder as the train set 
testix=index[0:300]
devix=index[300:600]
trainix=index[600:]

In [3]:
df=pd.read_csv('lowva\lowva-structured.csv')

In [4]:
devlist=df['pat_deid'][devix].tolist()

In [5]:
benstest=pd.read_csv('lowva\Test Set\lowva-testset.csv', header=None)

In [6]:
benstestlist=benstest[0].tolist()

In [7]:
ratinglist=list(set(devlist).union(set(benstestlist)))

In [8]:
dfnotes=pd.read_csv("lowva/lowva-notesconcat.csv")

In [9]:
dfnotestorate=dfnotes[dfnotes["pat_deid"].isin(ratinglist)]

In [10]:
dfnotestorate.to_csv('lowva/lowva-notestorate.csv', index=False)

In [11]:
for i in range(1): 
    print(dfnotestorate.iloc[i])

pat_deid                                                 2920
outcome                                                     1
note        Stanford University Hospital and Clinics  Opht...
Name: 5, dtype: object


In [12]:
benstestlist

[3104549,
 1266179,
 2728026,
 2089670,
 3202543,
 2486364,
 923881,
 3716264,
 729752,
 2937244,
 2938754,
 2212351,
 2016494,
 1463748,
 3368141,
 3385898,
 2258844,
 2292225,
 252089,
 1900281,
 2860374,
 2075173,
 1652453,
 3476261,
 2950574,
 1526192,
 272754,
 3750942,
 3447506,
 982386,
 3667669,
 3731979,
 627787,
 1410122,
 956797,
 1938018,
 2562110,
 2941753,
 2228006,
 3590434,
 3694422,
 1373207,
 3628717,
 427840,
 214659,
 1937733,
 2988114,
 139556,
 1785082,
 3429894,
 2338801,
 2211271,
 3751761,
 1935677,
 2083352,
 278693,
 1856770,
 2106154,
 2469628,
 1845189,
 416501,
 2813654,
 3446895,
 3814217,
 3667600,
 860439,
 2052264,
 962562,
 2237291,
 459497,
 240560,
 2045171,
 2838580,
 3130095,
 1660802,
 1894777,
 2847332,
 3342608,
 599248,
 3283472,
 1422196,
 1810064,
 136545,
 724939,
 125693,
 1486188,
 2719957,
 1002525,
 1026483,
 519046,
 2383315,
 3668611,
 2924532,
 2908825,
 3286835,
 905988,
 2866775,
 2790096,
 1773843,
 2338743,
 2219826,
 648804,
 38

In [13]:
ratinglist

[962562,
 1746947,
 131075,
 2586630,
 3668999,
 2266489,
 3295247,
 1970194,
 142354,
 990229,
 2666518,
 1373207,
 3347479,
 1002525,
 1331230,
 3750942,
 404511,
 173096,
 3303465,
 1735723,
 1312812,
 2921517,
 3467310,
 1913904,
 1218609,
 483378,
 3664945,
 2838580,
 3576889,
 1642555,
 2486332,
 3669052,
 675900,
 2562110,
 2845760,
 1014846,
 905288,
 2844744,
 1410122,
 627787,
 2600012,
 3366989,
 1597518,
 1423439,
 2353230,
 1266769,
 521294,
 2109523,
 780365,
 2988114,
 3203162,
 856155,
 2728026,
 2848858,
 2486364,
 2430047,
 3023967,
 393313,
 1042525,
 2146398,
 1719399,
 1127527,
 1717354,
 2298986,
 2898028,
 289901,
 3446895,
 234608,
 1520757,
 238710,
 3821687,
 3116152,
 553081,
 48250,
 2074747,
 2021501,
 1834112,
 364675,
 2339972,
 828548,
 1078406,
 961671,
 363656,
 3676298,
 402575,
 65679,
 1246354,
 232595,
 92308,
 3518621,
 2983070,
 1697952,
 79008,
 1947812,
 3649701,
 2222246,
 1409190,
 995496,
 3678374,
 3609764,
 920747,
 278693,
 3716264,
 2052

In [14]:
conn = sqlite3.connect('lowva/lowva.db')
conn.text_factory = str
cur = conn.cursor()

In [15]:
rated=pd.read_sql_query('''select * from humanrating''', conn)

In [16]:
len(rated[rated["pat_deid"].isin(devlist)])

102

In [17]:
len(rated[rated["pat_deid"].isin(benstestlist)])

102

In [18]:
testlist=df['pat_deid'][testix].tolist()

In [19]:
len(rated[rated["pat_deid"].isin(testlist)])

5

In [20]:
rated["rating"].mean()

0.595

In [22]:
#check to see how things are going so far 
dfagreement=pd.read_sql_query('''select humanrating.pat_deid, rating, outcome from humanrating, outcome
where humanrating.pat_deid=outcome.pat_deid''', conn)

In [23]:
from sklearn.metrics import classification_report
y_true = np.array(dfagreement["outcome"])
y_pred = np.array(dfagreement["rating"])
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.51      0.59       114
           1       0.53      0.73      0.61        86

    accuracy                           0.60       200
   macro avg       0.62      0.62      0.60       200
weighted avg       0.64      0.60      0.60       200



In [24]:
#confusion matrix 
pd.crosstab(dfagreement["rating"], dfagreement["outcome"])

outcome,0,1
rating,,
0,58,23
1,56,63


In [27]:
bensubset=dfagreement[dfagreement["pat_deid"].isin(benstestlist)]

In [28]:
y_true = np.array(bensubset["outcome"])
y_pred = np.array(bensubset["rating"])
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.67      0.49      0.57        57
           1       0.52      0.69      0.59        45

    accuracy                           0.58       102
   macro avg       0.59      0.59      0.58       102
weighted avg       0.60      0.58      0.58       102



In [193]:
conn.close()